## COVID-19 Data Modeling

In [ ]:
!bq --location=US mk --dataset covid_19_modeled

In [14]:
%%bigquery
create or replace table covid_19_modeled.Daily_Cases 
as select null as id, * from covid_19_staging.Daily_Cases

""


### Compute fingerprint of location fields (state, country) 

In [16]:
%%bigquery
update covid_19_modeled.Daily_Cases set id = FARM_FINGERPRINT(country) 
where state is null

""


In [17]:
%%bigquery
update covid_19_modeled.Daily_Cases set id = FARM_FINGERPRINT(concat(state, country)) 
where state is not null

""


In [18]:
%%bigquery
select id, state, country
from covid_19_modeled.Daily_Cases
order by state, country
limit 5

,id,state,country
0,8576431891811451300,None,Azerbaijan
1,8778414404485170876,None,Afghanistan
2,8778414404485170876,None,Afghanistan
3,8778414404485170876,None,Afghanistan
4,8778414404485170876,None,Afghanistan


### Split Daily_Cases table

In [24]:
%%bigquery
create or replace table covid_19_modeled.Location
as select distinct id, state, country, latitude, longitude
from covid_19_modeled.Daily_Cases

""


In [27]:
%%bigquery
select count(*) as distinct_location from covid_19_modeled.Location

,distinct_location
0,415


In [25]:
%%bigquery
create or replace table covid_19_modeled.Event
as select id as location_id, last_update, confirmed, deaths, recovered
from covid_19_modeled.Daily_Cases

""


In [28]:
%%bigquery
select count(*) as distinct_location from covid_19_modeled.Event

,distinct_location
0,4247


### Update confirmed, deaths, and recovered to 0 where null

In [29]:
%%bigquery
update covid_19_modeled.Event
set confirmed = 0 where confirmed is null

""


In [30]:
%%bigquery
update covid_19_modeled.Event 
set deaths = 0 where deaths is null

""


In [31]:
%%bigquery
update covid_19_modeled.Event 
set recovered = 0 where recovered is null

""


In [32]:
%run Event_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []


In [33]:
%run Location_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []


### Check primary key constraint on Event_Beam table

In [34]:
%%bigquery
select count(*) as total_event_count from covid_19_modeled.Event_Beam

,total_event_count
0,4247


In [35]:
%%bigquery
select count(*) from
(select distinct last_update, location_id as distinct_event_count from covid_19_modeled.Event_Beam)

,f0_
0,2598


In [43]:
%%bigquery
select last_update, location_id, count(*) as duplicate_count
from covid_19_modeled.Event_Beam
group by last_update, location_id
having count(*) > 1
limit 7

,last_update,location_id,duplicate_count
0,2020-02-26 23:53:02,8222489297432220498,3
1,2020-03-02 20:33:02,4279641640960423873,3
2,2020-03-02 20:23:16,979981601557933813,7
3,2020-02-12 14:43:03,4591735267635711941,19
4,2020-02-28 16:23:03,4279641640960423873,3
5,2020-03-02 20:23:16,-8827382212301428434,7
6,2020-03-02 01:33:02,-2675632486700738021,2


### TO DO: detect and remove duplicate location records in Beam

### Check primary key constraint on Location_Beam table

In [36]:
%%bigquery
select count(*) as total_location_count from covid_19_modeled.Location_Beam

,total_location_count
0,415


In [37]:
%%bigquery
select count(distinct id) as distinct_location_count from covid_19_modeled.Location_Beam

,distinct_location_count
0,289


In [40]:
%%bigquery
select id, city, state, country, longitude, latitude from covid_19_modeled.Location_Beam
order by city, state, country
limit 7

,id,city,state,country,longitude,latitude
0,8576431891811451300,None,None,Azerbaijan,None,None
1,8778414404485170876,None,None,Afghanistan,65,33
2,8778414404485170876,None,None,Afghanistan,None,None
3,-3566989785867701492,None,None,Algeria,1.6596,28.0339
4,-3566989785867701492,None,None,Algeria,None,None
5,-8827382212301428434,None,None,Andorra,1.5218,42.5063
6,-2806519937964814283,None,None,Argentina,-63.6167,-38.4161


### TO DO: remove duplicate location records in Beam 